![zad6](img/zad6.png)

In [146]:
import numpy as np
import pandas as pd

In [84]:
A = np.array(
    [
        [2, -3, 3, 2],
        [6, -9, -2, -1],
        [10, 3, -3, -2],
        [8, 6, 1, 3]
    ], dtype=float
)

b = np.array(
    [
        3,
        -4,
        3,
        -7
    ], dtype=float
)

In [85]:
def make_identity(matrix):
    # перебор строк в обратном порядке 
    for nrow in range(len(matrix)-1,0,-1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            upper_row -= factor*row
    return matrix

In [86]:
def gaussPivotFunc(matrix):
    for nrow in range(len(matrix)):
        # nrow равен номеру строки
        # np.argmax возвращает номер строки с максимальным элементом в уменьшенной матрице
        # которая начинается со строки nrow. Поэтому нужно прибавить nrow к результату
        pivot = nrow + np.argmax(abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            # swap
            # matrix[nrow], matrix[pivot] = matrix[pivot], matrix[nrow] - не работает.
            # нужно переставлять строки именно так, как написано ниже
            matrix[[nrow, pivot]] = matrix[[pivot, nrow]]
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            # почти нуль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
            raise ValueError(f"Матрица несовместна. Максимальный элемент в столбце {nrow}: {divider:.3g}")
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    # приводим к диагональному виду
    make_identity(matrix)
    return matrix

In [149]:
matrix = np.column_stack((A.copy(), b.copy()))
matrix = gaussPivotFunc(matrix)
pd.DataFrame(matrix)

,0,1,2,3,4
0,1.0,0.0,0.0,0.0,0.500000
1,0.0,1.0,0.0,0.0,0.333333
2,0.0,0.0,1.0,0.0,5.000000
3,0.0,0.0,0.0,1.0,-6.000000


In [165]:
x = np.array([
    0,
    0,
    0,
    0
], dtype=float)


for row in range(3, -1, -1):
    x[row] = matrix[row][-1] - sum(x[i]*matrix[row][i] for i in range(4))
    

print('Gauss method')
pd.DataFrame(x, columns=['x'], index=[x for x in range(1, 5)])

Gauss method


,x
1,0.500000
2,0.333333
3,5.000000
4,-6.000000


In [157]:
def Kram(A, b):
    x = np.zeros(len(A))
    det = np.linalg.det(A)

    if det == 0:
        raise ValueError

    for column in range(len(A[0])):
        matrix = A.copy()
        for row in range(len(A)):
            matrix[row][column] = b[row]
        det_x = np.linalg.det(matrix)
        x[column] = det_x/det

    return x

In [164]:
x = Kram(A, b)
print('Kramer method')
pd.DataFrame(x, columns=['x'], index=[x for x in range(1, 5)])

Kramer method


,x
1,0.500000
2,0.333333
3,5.000000
4,-6.000000


In [91]:
def inverse_matrix(A, b):
    A_inv = np.linalg.inv(A)

    x = np.dot(A_inv, b)

    return x

In [159]:
x = inverse_matrix(A, b)

pd.DataFrame(x, columns=['x'], index=[x for x in range(1, 5)])

,x
1,0.500000
2,0.333333
3,5.000000
4,-6.000000


In [162]:
def lu_decomposition_pivot(A):
    A = A.copy()
    n = len(A)
    L = np.zeros_like(A, dtype=float)
    U = np.zeros_like(A, dtype=float)
    P = np.eye(n)

    for i in range(n):
        # Pivoting
        max_row = np.argmax(abs(A[i:, i])) + i
        if A[max_row, i] == 0:
            raise ValueError("Matrix is singular.")
        if max_row != i:
            A[[i, max_row]] = A[[max_row, i]]
            P[[i, max_row]] = P[[max_row, i]]
            L[[i, max_row], :i] = L[[max_row, i], :i]

        L[i, i] = 1
        for j in range(i, n):
            U[i, j] = A[i, j] - L[i, :i] @ U[:i, j]
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - L[j, :i] @ U[:i, i]) / U[i, i]

    return P, L, U


def forward_substitution(L, b):
    """
    Solve Ly = b for y using forward substitution
    """
    n = len(b)
    y = np.zeros(n)
    
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    return y

def backward_substitution(U, y):
    """
    Solve Ux = y for x using backward substitution
    """
    n = len(y)
    x = np.zeros(n)
    
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    
    return x

def lu_solve(A, b):
    """
    Solve system Ax = b using LU decomposition
    """
    P, L, U = lu_decomposition_pivot(A)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

In [163]:
x = lu_solve(A, b)
print('LU decomposition method')

pd.DataFrame(x, columns=['x'], index=[x for x in range(1, 5)])

LU decomposition method


,x
1,0.500000
2,0.333333
3,5.000000
4,-6.000000
